# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# Read csv files from the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the total number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Apache Cassandra Data Modelling

At this point CSV file titled <font color=red>event_datafile_new.csv</font> should have been generated and located within the current workspace directory. The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Connect to local Cassandra instance (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify \
                     WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to sparkify keyspace 
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Create data models for queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query 1
> Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To be able to query songplays with specified `session_id` and `item_in_session`, each of them is set to be the partition key and clustering column correspondingly. As the `session_id` is spread among users uniquely, it can be used as the only partition key to distribute the data accross Cassandra partitions evenly. Compounded with `item_in_session`, it makes unique entry for each songplay item. 

In [8]:
## Create table
query = """CREATE TABLE IF NOT EXISTS songplays_by_session (
             session_id int, item_in_session int, 
             artist_name text, song_title text, song_length float, 
             PRIMARY KEY (session_id, item_in_session)
           )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
## Insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO songplays_by_session (
                     session_id, item_in_session, artist_name, song_title, song_length
                   )
                """
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign column element from csv row
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## Create SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""SELECT artist_name, song_title, song_length 
                              FROM songplays_by_session 
                              WHERE session_id=338 AND item_in_session=4;
                           """)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2

> Query 2: Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In this query, to make unique songplay item entries we need to create composite keys which consists of `user_id`, `session_id`, and `item_in_session`. While we can assign `user_id` as the only partition key, it is better to spread the data partitions based on non specific user data. So in this case `user_id` and `session_id` are the best suit for partition keys. `item_in_session` is assigned as the clustering column so the data is sorted.  

In [11]:
## Create table
query = """CREATE TABLE IF NOT EXISTS songplays_by_userid_session (
             user_id int, session_id int, item_in_session int, 
             user_first_name text, user_last_name text, 
             artist_name text, song_title text,
             PRIMARY KEY ((user_id, session_id), item_in_session)
           )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [12]:
## Insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO songplays_by_userid_session (
                     user_id, session_id, item_in_session,
                     user_first_name, user_last_name, 
                     artist_name, song_title
                   )
                """
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign column element from csv row
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [13]:
## Create SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""SELECT artist_name, song_title, user_first_name, user_last_name 
                              FROM songplays_by_userid_session 
                              WHERE user_id=10 AND session_id=182;
                           """)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.user_first_name, row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3

> Query 3: Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In this case, the query only focus on the user information in the songplay data based on specified `song_title`. Duplicate song entry from one user is still OK. Therefore, we can just choose `song_title` as the partition key and `user_id` as clustering column. 

In [14]:
## Create table
query = """CREATE TABLE IF NOT EXISTS songplays_by_song (
             song_title text, user_id int, user_first_name text, user_last_name text,
             PRIMARY KEY (song_title, user_id)
           )
        """
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [15]:
## Insert into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO songplays_by_song (
                     song_title, user_id, user_first_name, user_last_name
                   )
                """
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign column element from csv row
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## Create SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("""SELECT user_first_name, user_last_name 
                              FROM songplays_by_song 
                              WHERE song_title='All Hands Against His Own';
                           """)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Cleanup

Drop the tables before closing out the sessions

In [17]:
try:
    session.execute("DROP TABLE IF EXISTS songplays_by_session")
    session.execute("DROP TABLE IF EXISTS songplays_by_userid_session")
    session.execute("DROP TABLE IF EXISTS songplays_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()